Importamos las librerìas necesarias

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd


# Version 1

In [ ]:
def scrape_books_list(url, num_books):
    # URL base de la página
    base_url = "https://ww3.lectulandia.com"
    page_num = 1
    books_data = []

    while len(books_data) < num_books:
        # URL de la página actual
        current_url = f"{base_url}/book/page/{page_num}/"

        response = requests.get(current_url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Encontrar todos los elementos que contienen información de los libros
        book_elements = soup.find_all('article', class_='card')

        # Iterar sobre los elementos de los libros
        for book in book_elements:
            if len(books_data) >= num_books:
                break

            # Obtener el título del libro
            title = book.find('a', class_='title').text.strip()

            # Obtener el autor del libro
            author = book.find('div', class_='subdetail').a.text.strip()

            # Obtener la URL del libro
            book_url = base_url + book.find('a', class_='title')['href']

            # Realizar la solicitud GET a la página del libro para obtener el género
            book_response = requests.get(book_url)
            book_soup = BeautifulSoup(book_response.text, 'html.parser')

            # Encontrar el elemento que contiene el género del libro
            genre_elements = book_soup.find('div', id='genero')

            # Encontrar todos los enlaces <a> dentro del elemento
            genre_links = genre_elements.find_all('a') if genre_elements else []

            # Obtener el o los género/s del libro
            genres = ", ".join([link.text.strip() for link in genre_links])

            # Obtener la descripción del libro
            description_element = book.find('div', class_='description')
            description = description_element.p.text.strip() if description_element else "Sin descripción"

            # Agregar los datos del libro a la lista
            books_data.append([title, author, description, genres])

        # Incrementar el número de página
        page_num += 1

        # Crear un DataFrame a partir de la lista de libros
        books_df = pd.DataFrame(books_data, columns=['Title', 'Author', 'Description', 'Genres'])

    return books_df

In [ ]:
# Llamada a la función
url = "https://ww3.lectulandia.com/"
books = scrape_books_list(url, 500)

In [ ]:
books

,Title,Author,Description,Genres
0,Sueñan con ser como nosotras,Jessica Goodman,"Carismáticas, intocables, destinadas a triunfa...","Intriga, Juvenil, Novela"
1,El descontento,Beatriz Serrano,"El descontento es la historia de Marisa, una m...","Novela, Realista"
2,Palabra de reina,Gema Bonnín,La novela sobre la vida y el poder de Catalina...,"Histórico, Novela"
3,Irresistible,Jose de la Rosa,"Adam Baxley, el disoluto heredero del conde de...","Histórico, Novela, Romántico"
4,La criatura del deseo,Andrea Camilleri,"Basándose en personajes y hechos reales, Andre...","Drama, Histórico, Novela"
...,...,...,...,...
495,Roxy,Jarrod Shusterman,Falta poco para que terminen de construir la a...,"Drama, Novela"
496,Hijos de Gael,Rodrigo Costoya,"En la guerra de la razón contra el dogma, un n...","Histórico, Novela"
497,Calamar : casi película policiaca en tres jorn...,Pedro Muñoz Seca,Calamar es una comedia teatral del autor Pedro...,"Humor, Teatro"
498,Esos besos robados son míos,Marian Arpa,Grace se cruza en la vida de Hans en el peor m...,"Erótico, Humor, Novela, Romántico"


In [ ]:
df['Combined'] = df['Title'] + ' ' + df['Description'] + ' ' + df['Genres']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Función para obtener recomendaciones directas basadas en la respuesta del usuario
def direct_recommendation(df, user_input):
    # Combinar título, descripción y géneros para mejorar la clasificación
    df['Combined'] = df['Title'] + ' ' + df['Description'] + ' ' + df['Genres']

    # Vectorización del texto
    vectorizer = TfidfVectorizer(stop_words='spanish')
    tfidf_matrix = vectorizer.fit_transform(df['Combined'])

    # Vectorización de la entrada del usuario
    user_input_tfidf = vectorizer.transform([user_input])

    # Calcular la similitud del coseno entre la entrada del usuario y el conjunto de libros
    cosine_similarities = cosine_similarity(user_input_tfidf, tfidf_matrix).flatten()

    # Obtener los índices de los libros con mayor similitud
    related_docs_indices = cosine_similarities.argsort()[-3:][::-1]

    # Obtener los libros recomendados
    recommendations = df.iloc[related_docs_indices]

    return recommendations

def display_recommendations(recommendations):
    if recommendations.empty:
        print("No se encontraron libros que coincidan con tu búsqueda.")
        return

    print("Recomendaciones basadas en tus intereses:")
    for index, row in recommendations.iterrows():
        print(f"Título: {row['Title']}")
        print(f"Autor: {row['Author']}")
        print(f"Descripción: {row['Description']}")
        print(f"Géneros: {row['Genres']}")
        print("-" * 40)

# Función principal para interactuar con el usuario
def main():
    while True:
        print("\nOpciones de recomendación:")
        print("1. Recomendación Directa")
        print("2. Elección por Autor")
        print("3. Salir")

        choice = input("Selecciona una opción (1/2/3): ")

        if choice == '1':
            user_input = input("¿Qué tienes ganas de leer hoy? ")
            recommendations = direct_recommendation(books, user_input)
            display_recommendations(recommendations)
        elif choice == '2':
            author_name = input("Ingrese el nombre del autor: ")
            recommendations = search_by_author(books, author_name)
            display_recommendations(recommendations)
        elif choice == '3':
            print("¡Hasta luego!")
            break
        else:
            print("Opción no válida. Por favor, selecciona 1, 2 o 3.")


In [ ]:
import pandas as pd
import numpy as np

def search_by_author(df, author_name):
    # Filtrar libros por el autor especificado
    author_books = df[df['Author'].str.contains(author_name, case=False, na=False)]

    if author_books.empty:
        print("No se encontraron libros del autor especificado.")
        return None

    # Si hay más de un libro, aleatorizar y seleccionar los más relevantes
    if len(author_books) > 1:
        author_books = author_books.sample(frac=1).reset_index(drop=True)  # Aleatorizar

    # Seleccionar los dos primeros libros como los más relevantes
    first_recommendation = author_books.iloc[0:2]

    # Si hay más de dos libros, seleccionar uno más del segundo grupo más relevante
    second_recommendation = None
    if len(author_books) > 2:
        second_recommendation = author_books.iloc[2]

    # Preparar las recomendaciones
    recommendations = {
        'first': first_recommendation,
        'second': second_recommendation
    }

    return recommendations

def display_recommendations(recommendations):
    if not recommendations:
        return

    print("Recomendaciones basadas en el autor especificado:")
    print("\nPrimer grupo de recomendaciones:")
    for index, row in recommendations['first'].iterrows():
        print(f"Título: {row['Title']}")
        print(f"Autor: {row['Author']}")
        print(f"Descripción: {row['Description']}")
        print(f"Géneros: {row['Genres']}")
        print("-" * 40)

    if recommendations['second'] is not None:
        print("\nSegunda recomendación:")
        row = recommendations['second']
        print(f"Título: {row['Title']}")
        print(f"Autor: {row['Author']}")
        print(f"Descripción: {row['Description']}")
        print(f"Géneros: {row['Genres']}")
        print("-" * 40)

# Ejemplo de uso:
author_name = input("Ingrese el nombre del autor: ")
recommendations = search_by_author(books, author_name)
display_recommendations(recommendations)


Ingrese el nombre del autor: Jose de la Rosa
Recomendaciones basadas en el autor especificado:

Primer grupo de recomendaciones:
Título: Irresistible
Autor: Jose de la Rosa
Descripción: Adam Baxley, el disoluto heredero del conde de Dunwich, se encuentra en una encrucijada desesperada. Su padre le impone un matrimonio y un heredero en un plazo implacable. Si fracasa, perderá su posición, fortuna y título, cayendo en la desgracia social. La elegida para ser su esposa no puede ser peor, pues se trata de […]
Géneros: Histórico, Novela, Romántico
----------------------------------------


In [ ]:
# Funcion de similitud de coseno
def cosine_similarity(A, B):
    """
    Calcula la similitud del coseno entre dos vectores A y B.

    Parámetros:
    - A, B: Vectores de entrada.

    Retorna:
    - Similitud del coseno entre A y B.
    """
    dot_product = np.dot(A, B)
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)

    return dot_product / (norm_A * norm_B)

# Ejemplo de uso:
vector_A = np.array([1, 2, 3])
vector_B = np.array([4, 5, 6])

print(cosine_similarity(vector_A, vector_B))

In [ ]:
## CON FAST TEXT (QUE SEGUN LA TEORIA ES EL MEJOR)
## O PODRIA SER ELMO TAMBIEN
## ESTO LO COPIE DE LA TEORIA
!pip install fasttext
!pip install huggingface_hub

import fasttext
from huggingface_hub import hf_hub_download

# Descargamos el modelo FastText para español (Aprox. 7Gb)
# https://huggingface.co/facebook/fasttext-es-vectors
model_path = hf_hub_download(repo_id="facebook/fasttext-es-vectors", filename="model.bin")
model = fasttext.load_model(model_path)

import numpy as np

# Función para calcular la similitud del coseno
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

neighbors = model.get_nearest_neighbors("perro", k=5)
print("neighbors:", neighbors)

# Calcular la similitud del coseno
similarity = cosine_similarity(model.get_word_vector("rey"), model.get_word_vector("reina"))
print(f"\nSimilitud del coseno entre 'rey' y 'reina': {similarity}")

# Calcular la similitud del coseno
similarity = cosine_similarity(model.get_word_vector("derecha"), model.get_word_vector("izquierda"))
print(f"\nSimilitud del coseno entre 'derecha' y 'izquierda': {similarity}")

# Calcular la similitud del coseno
similarity = cosine_similarity(model.get_word_vector("pentagrama"), model.get_word_vector("casualidad"))
print(f"\nSimilitud del coseno entre 'pentagrama' y 'casualidad': {similarity}")

# Obtener la vectorización de subpalabras para una palabra específica
word = 'banco'
subwords, subword_indices = model.get_subwords(word)

# Imprimir las subpalabras y sus vectores
for subword, idx in zip(subwords, subword_indices):
    print(f'Subpalabra: {subword}')
    vector = model.get_input_vector(idx)


## ESTO ES LO QUE ME TIRA CHATGPT
import pandas as pd
import numpy as np
import fasttext
import fasttext.util
from sklearn.metrics.pairwise import cosine_similarity

# Función para entrenar el modelo de FastText
def train_fasttext_model(df):
    # Crear un archivo temporal con el corpus de texto
    with open("corpus.txt", "w", encoding="utf-8") as f:
        for index, row in df.iterrows():
            combined_text = f"{row['Title']} {row['Description']} {row['Genres']}"
            f.write(combined_text + "\n")

    # Entrenar el modelo de FastText
    model = fasttext.train_unsupervised("corpus.txt", model='skipgram')

    return model

# Función para obtener el vector de una frase utilizando FastText
def get_text_vector(model, text):
    words = text.split()
    word_vectors = [model.get_word_vector(word) for word in words if word in model]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros((model.get_dimension(),))

# Función para obtener recomendaciones directas basadas en la respuesta del usuario
def direct_recommendation(df, model, user_input):
    # Vectorizar la entrada del usuario
    user_input_vector = get_text_vector(model, user_input)

    # Vectorizar los textos combinados de los libros
    df['Combined'] = df['Title'] + ' ' + df['Description'] + ' ' + df['Genres']
    df['Vector'] = df['Combined'].apply(lambda x: get_text_vector(model, x))

    # Calcular la similitud del coseno entre la entrada del usuario y el conjunto de libros
    df['Similarity'] = df['Vector'].apply(lambda x: cosine_similarity([user_input_vector], [x]).flatten()[0])

    # Obtener los libros con mayor similitud
    recommendations = df.sort_values(by='Similarity', ascending=False).head(3)

    return recommendations

def display_recommendations(recommendations):
    if recommendations.empty:
        print("No se encontraron libros que coincidan con tu búsqueda.")
        return

    print("Recomendaciones basadas en tus intereses:")
    for index, row in recommendations.iterrows():
        print(f"Título: {row['Title']}")
        print(f"Autor: {row['Author']}")
        print(f"Descripción: {row['Description']}")
        print(f"Géneros: {row['Genres']}")
        print("-" * 40)

# Función principal para interactuar con el usuario
def main():
    while True:
        print("\nOpciones de recomendación:")
        print("1. Recomendación Directa")
        print("2. Elección por Autor")
        print("3. Salir")

        choice = input("Selecciona una opción (1/2/3): ")

        if choice == '1':
            user_input = input("¿Qué tienes ganas de leer hoy? ")
            recommendations = direct_recommendation(books_df, fasttext_model, user_input)
            display_recommendations(recommendations)
        elif choice == '2':
            author_name = input("Ingrese el nombre del autor: ")
            recommendations = search_by_author(books_df, author_name)
            display_recommendations(recommendations)
        elif choice == '3':
            print("¡Hasta luego!")
            break
        else:
            print("Opción no válida. Por favor, selecciona 1, 2 o 3.")

# Ejemplo de uso
if __name__ == "__main__":
    # Cargar los datos (esto es solo un ejemplo, asegúrate de cargar tu DataFrame correctamente)
    # books_df = pd.read_csv('path_to_your_books_data.csv') # Cargar el DataFrame desde un archivo CSV

    # Para fines de demostración, aquí hay un ejemplo de un DataFrame simulado:
    books_data = {
        'Title': ['Libro1', 'Libro2', 'Libro3'],
        'Author': ['Autor1', 'Autor2', 'Autor3'],
        'Description': ['Descripción1', 'Descripción2', 'Descripción3'],
        'Genres': ['Género1', 'Género2', 'Género3']
    }
    books_df = pd.DataFrame(books_data)

    # Entrenar el modelo de FastText
    fasttext_model = train_fasttext_model(books_df)

    main()


In [ ]:
## O PODRIA SER ELMO TAMBIEN
## ESTO LO COPIE DE LA TEORIA

!pip install elmoformanylangs
# Descarga modelo en español
!wget http://vectors.nlpl.eu/repository/11/145.zip
!unzip 145.zip -d elmo_es
# Fix para evitar error en Colab (https://github.com/HIT-SCIR/ELMoForManyLangs/issues/100)
!pip uninstall overrides
!pip install overrides==3.1.0

# Cargamos el modelo desde la carpeta donde fue descomprimido:
from elmoformanylangs import Embedder

# Ruta al directorio del modelo descargado
model_dir = '/content/elmo_es'

# Inicializar el modelo
e = Embedder(model_dir)

# Y aquí probaremos como ELMo nos permite obtener diferentes embeddings según el contexto:
# Lista de frases que contienen la palabra "banco" en diferentes contextos
sents = [
    ['Fui', 'al', 'banco', 'a', 'retirar', 'dinero'],  # Contexto financiero
    ['Me', 'senté', 'en', 'el', 'banco', 'del', 'parque'],  # Contexto de asiento
    ['El', 'jugador', 'estuvo', 'en', 'el', 'banco', 'de', 'suplentes']  # Contexto deportivo
]

# Le ponemos nombres a los contextos, para luego identificarlos
contexts = ['financiero', 'asiento', 'deportivo']

# Obtener embeddings
embeddings = e.sents2elmo(sents)

# Imprimir los embeddings para la palabra "banco" en cada contexto
for idx, (sentence, embedding) in enumerate(zip(sents, embeddings)):
    # El embedding de la palabra "banco" se encuentra en la posición donde aparece en cada frase.
    position = sentence.index('banco')

    # Obtener el embedding de la palabra "banco" en esa posición
    banco_embedding = embedding[position]

    # Determinar el contexto basado en la frase
    context = contexts[idx]

    print(f"Embedding para 'banco' en contexto '{context}':\n{banco_embedding}\n")

## ESTO ME LO TIRA CHATGPT
pip install tensorflow tensorflow_hub scikit-learn pandas numpy
import pandas as pd
import numpy as np
import tensorflow_hub as hub
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity

# Función para obtener las representaciones de ELMo
def embed_text(text_list):
    embed = hub.load("https://tfhub.dev/google/elmo/3")
    embeddings = embed(text_list, signature="default", as_dict=True)["default"]
    return embeddings

# Función para obtener recomendaciones directas basadas en la respuesta del usuario
def direct_recommendation(df, user_input):
    # Combinar título, descripción y géneros para mejorar la clasificación
    df['Combined'] = df['Title'] + ' ' + df['Description'] + ' ' + df['Genres']

    # Obtener las representaciones de ELMo para los textos combinados de los libros
    combined_texts = df['Combined'].tolist()
    book_embeddings = embed_text(combined_texts).numpy()

    # Obtener la representación de ELMo para la entrada del usuario
    user_input_embedding = embed_text([user_input]).numpy()

    # Calcular la similitud del coseno entre la entrada del usuario y el conjunto de libros
    cosine_similarities = cosine_similarity(user_input_embedding, book_embeddings).flatten()

    # Obtener los índices de los libros con mayor similitud
    related_docs_indices = cosine_similarities.argsort()[-3:][::-1]

    # Obtener los libros recomendados
    recommendations = df.iloc[related_docs_indices]

    return recommendations

def display_recommendations(recommendations):
    if recommendations.empty:
        print("No se encontraron libros que coincidan con tu búsqueda.")
        return

    print("Recomendaciones basadas en tus intereses:")
    for index, row in recommendations.iterrows():
        print(f"Título: {row['Title']}")
        print(f"Autor: {row['Author']}")
        print(f"Descripción: {row['Description']}")
        print(f"Géneros: {row['Genres']}")
        print("-" * 40)

# Función principal para interactuar con el usuario
def main():
    while True:
        print("\nOpciones de recomendación:")
        print("1. Recomendación Directa")
        print("2. Elección por Autor")
        print("3. Salir")

        choice = input("Selecciona una opción (1/2/3): ")

        if choice == '1':
            user_input = input("¿Qué tienes ganas de leer hoy? ")
            recommendations = direct_recommendation(books_df, user_input)
            display_recommendations(recommendations)
        elif choice == '2':
            author_name = input("Ingrese el nombre del autor: ")
            recommendations = search_by_author(books_df, author_name)
            display_recommendations(recommendations)
        elif choice == '3':
            print("¡Hasta luego!")
            break
        else:
            print("Opción no válida. Por favor, selecciona 1, 2 o 3.")

# Ejemplo de uso
if __name__ == "__main__":
    # Cargar los datos (esto es solo un ejemplo, asegúrate de cargar tu DataFrame correctamente)
    # books_df = pd.read_csv('path_to_your_books_data.csv') # Cargar el DataFrame desde un archivo CSV

    # Para fines de demostración, aquí hay un ejemplo de un DataFrame simulado:
    books_data = {
        'Title': ['Libro1', 'Libro2', 'Libro3'],
        'Author': ['Autor1', 'Autor2', 'Autor3'],
        'Description': ['Descripción1', 'Descripción2', 'Descripción3'],
        'Genres': ['Género1', 'Género2', 'Género3']
    }
    books_df = pd.DataFrame(books_data)

    main()


# Version 2

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Crear un DataFrame vacío con las columnas especificadas
df_libros = pd.DataFrame(columns=['Genero', 'Titulo', 'Autor', 'Descripcion'])

# Obtener el contenido HTML de la página principal
url_index = 'https://ww3.lectulandia.com'
response_index = requests.get(url_index)
html_content_index = response_index.text
soup_index = BeautifulSoup(html_content_index, 'html.parser')

# Armamos un objeto vacio donde vamos a ir almacenando los libros
data = {'Genero': [],
        'Titulo': [],
        'Autor': [],
        'Descripcion': []}

# Lista de generos a buscar
generos = ["ficcion", "historia", "misterio", "infantil", "arte", "economia", "deporte", "poesia", "religion", "terror"]

# Encontramos el tag <section> con id 'secgenero' que es el que contiene a todos los generos
soup_generos = soup_index.find('section', id='secgenero')

# Iteramos la lista de generos, y obetenemos el enlace correspondiente
for genero in generos:
  # Buscamos el tag <a> que en e href contiene la url correspondiente
  soup = soup_generos.find('a', href=f'/genero/{genero}/')
  if soup:
    # Construir el enlace completo y armamos un nuevo objeto soup
    url_genero = url_index + soup["href"]
    response_genero = requests.get(url_genero)
    html_content_genero = response_genero.text
    soup_genero = BeautifulSoup(html_content_genero, 'html.parser')
    # Extraemos 10 libros
    libros = soup_genero.find_all('article', class_='card')[:10]
    # Por cada libro buscamos el titulo, autor y descripcion y lo almacenamos en data
    # for i in range(10):
    for libro in libros:
      # libro = libros[i]
      detalle = libro.find('div', class_='details')
      data['Genero'].append(genero)
      titulo = detalle.find('a', class_ = 'title').text.strip()
      data['Titulo'].append(titulo)
      autor = libro.find('div', class_='subdetail').text.strip()
      data['Autor'].append(autor)
      descripcion = libro.find('div', class_='description').text.strip()
      data['Descripcion'].append(descripcion)

df_libros = pd.DataFrame(data)


In [ ]:
df_libros

,Genero,Titulo,Autor,Descripcion
0,ficcion,Carta al padre,Franz Kafka,He aquí el inicio de una carta que constituye ...
1,ficcion,La noche de Walburga,Gustav Meyrink,1914. La decadente Praga del imperio austrohún...
2,ficcion,El dinosaurio,Augusto Monterroso,"Microrrelato del autor hondureño, uno de los m..."
3,ficcion,Capitanes intrépidos,Rudyard Kipling,Escrita durante su periodo de residencia en Es...
4,ficcion,La casa solariega,Henryk Sienkiewicz,En La casa solariega (también conocida como La...
...,...,...,...,...
95,terror,Mystic Topaz,Pilar Pedraza,Cuando abrimos un libro de Pilar Pedraza a men...
96,terror,Corazón caníbal,Miguel Cané,"Claudia Castañeda, joven mexicana en Estados U..."
97,terror,Océanos de tiempo,Jerónimo Tristante,"Madrid, 1892. El prestigioso psiquiatra August..."
98,terror,Los recuerdos del hombre del saco,Goretti Irisarri · Jose Gil Romero,Un periodista está realizando una serie de ent...


In [ ]:
df_libros.groupby("Genero").count()

,Titulo,Autor,Descripcion
Genero,,,
arte,10,10,10
deporte,10,10,10
economia,10,10,10
ficcion,10,10,10
historia,10,10,10
infantil,10,10,10
misterio,10,10,10
poesia,10,10,10
religion,10,10,10


In [ ]:
df_libros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Genero       100 non-null    object
 1   Titulo       100 non-null    object
 2   Autor        100 non-null    object
 3   Descripcion  100 non-null    object
dtypes: object(4)
memory usage: 3.2+ KB


In [ ]:
df_libros['Titulo'].value_counts()

Titulo
La invención del cuerpo: arte y erotismo en el mundo clásico    2
Carta al padre                                                  1
Yambógrafos griegos                                             1
Poesía epigráfica latina II                                     1
Poesía epigráfica latina I                                      1
                                                               ..
La historia de Ernesto                                          1
Cuentos Montessori: Para crecer felices                         1
Una voz en la niebla                                            1
Las viudas de los jueves                                        1
Nieve Blanca                                                    1
Name: count, Length: 99, dtype: int64